In [85]:
# for setting-up, need to set-up nodejs because jupyterlab get extensions from nmp packages
# conda install -c conda-forge nodejs
#!jupyter nbextension enable --py widgetsnbextension
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
# (may need to restart your lab)

In [86]:
!jupyter labextension list
!jupyter nbextension list

JupyterLab v2.2.6
No installed extensions
Known nbextensions:
  config dir: C:\Users\nguye\.jupyter\nbconfig
    notebook section
      jupyter-js-widgets/extension enabled 
  config dir: C:\Users\nguye\anaconda3\envs\sem-pysot-37\etc\jupyter\nbconfig
    notebook section
      jupyter-webrtc/extension enabled 
      jupyter-js-widgets/extension enabled 


      - Validating: ok
      - Validating: ok
      - Validating: ok


In [87]:
import numpy as np
import cv2
import matplotlib
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import seaborn as sns

from ipywidgets import interact, interactive, fixed, interact_manual, Video, Image
import ipywidgets as widgets
from IPython.display import display

# Only video written by skvideo can be read in notebook - weird???
# import skvideo
# ffmpeg_path = 'C:/Users/nguye/ffmpeg-4.3.1-2020-10-01-full_build/bin'
# skvideo.setFFmpegPath(ffmpeg_path)
# import skvideo.io
# print("FFmpeg path: {}".format(skvideo.getFFmpegPath()))

In [88]:
# Example opening mp4 video interactively

outputdata = np.random.random(size=(100, 480, 680, 3)) * 255
outputdata = outputdata.astype(np.uint8)

skvideo.io.vwrite("outputvideo.mp4", outputdata)
video_widget = Video.from_file('outputvideo.mp4')
video_widget

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x01\xb3Ajmdat\x00\x00\…

In [89]:
import os
import sys
sys.path.append('../pysot')
from run_sem_with_features import preprocess_skel

skel_df = preprocess_skel('output/skel/4.4.5_skel_features.csv')

In [90]:
from utils import CV2VideoReader, FrameWrapper
import cv2
cv2_video_reader = CV2VideoReader('data/small_videos/4.4.5_C1_trim.mp4')
buffer_frames = dict()
frame_id = -1
while cv2_video_reader.capture.isOpened():
    frame_id += 1
    ret, frame = cv2_video_reader.read_frame()
    if frame_id > 10000:
        break
    if frame_id % 10 == 0:
        frame_wrapper = FrameWrapper(frame)
        frame_wrapper.put_text(f'FrameID: {frame_id}')
        buffer_frames[frame_id] = cv2.resize(frame_wrapper.frame, (480, 270))

In [96]:
# Example interactive widgets
from IPython.display import display
from ipywidgets import Image
import PIL.Image
@interact(index=(110, 10000, 10))
def one_feature(index=110):
    plt.ylim([0, 1])
    plt.plot(skel_df.loc[110: index, 'J0_3D_X'])
    plt.ylabel('J0_3D_X')
    plt.xlabel('FrameID')
    plt.scatter(index, skel_df.loc[index, 'J0_3D_X'], color='red')
    display(PIL.Image.fromarray(buffer_frames[index][:, :, ::-1]))
    

@interact(index=(110, 10000, 10))
def two_feature(index=110):
    plt.ylim([0, 1])
    plt.xlim([0, 1])
    plt.plot(skel_df.loc[102:index, 'J0_3D_X'], skel_df.loc[102: index, 'J11_3D_X'])
    plt.xlabel('J0_3D_X')
    plt.ylabel('J11_3D_X')
    plt.scatter(skel_df.loc[index, 'J0_3D_X'], skel_df.loc[index, 'J11_3D_X'], color='red')
    display(PIL.Image.fromarray(buffer_frames[index][:, :, ::-1]))

interactive(children=(IntSlider(value=110, description='index', max=10000, min=110, step=10), Output()), _dom_…

interactive(children=(IntSlider(value=110, description='index', max=10000, min=110, step=10), Output()), _dom_…

In [ ]:
# Example viewing multiple images
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

images = []
for img_path in glob.glob('output/1042/*.jpg'):
    images.append(mpimg.imread(img_path))

plt.figure(figsize=(20,20))
columns = 5
for i, image in enumerate(images):
    plt.subplot(len(images) / columns + 1, columns, i + 1)
    plt.imshow(image)